In [3]:
!pip install transformers
import fitz  # PyMuPDF
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

DEPRECATION: Loading egg at c:\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [4]:
!python -m spacy download en_core_web_sm
nlp=spacy.load('en_core_web_sm', disable=['ner', 'parser'])

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.1/12.8 MB 1.7 MB/s eta 0:00:08
      --------------------------------------- 0.2/12.8 MB 2.8 MB/s eta 0:00:05
     - -------------------------------------- 0.4/12.8 MB 3.4 MB/s eta 0:00:04
     - -------------------------------------- 0.6/12.8 MB 3.9 MB/s eta 0:00:04
     -- ------------------------------------- 0.9/12.8 MB 3.9 MB/s eta 0:00:04
     --- ------------------------------------ 1.1/12.8 MB 4.2 MB/s eta 0:00:03
     ---- ----------------------------------- 1.4/12.8 MB 4.3 MB/s eta 0:00:03
     ---- ----------------------------------- 1.6/12.8 MB 4.4 MB/s eta 0:00:03
     ----- ---------------------------------- 1.8/12.8 MB 4.5 MB/s eta 0:00:03
     ------ --------------------------------- 2.1/12.8 MB 4.8 MB/s eta 0:00:03
     ------- -------------------------------- 2.4/12.8 MB 5.0 MB/s eta 0:00:03
     -------- ------------------------------- 2.8/12.8 MB 5

DEPRECATION: Loading egg at c:\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [5]:
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page_num in range(doc.page_count):
        page = doc.load_page(page_num)
        if page_num!=1 and page_num!=2 and page_num!=0:
            text += page.get_text()
    return text

pdf_path = "C:\\Users\\NITRO\\Downloads\\1. Robinson Crusoe Author Daniel Defoe-compressed.pdf"

In [6]:
from transformers import pipeline

# Load a pre-trained model for question answering
model_name = "distilbert/distilbert-base-cased-distilled-squad"

# Create the question-answering pipeline with the specified model and revision
question_answering_model = pipeline("question-answering", model=model_name)


c:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
def sentence(pdf_text):
    sentence=pdf_text.split('.')
    for i in range(len(sentence)):
        sentence[i]=sentence[i].replace('\n',' ' )
    return sentence

In [8]:
def tokenize(doc):
    tokenized_sentences = []
    for d in doc:
        tokenized_sentence = ""
        for t in nlp(d):
            if not t.is_stop and not t.is_punct and len(t.text) > 2 and not t.is_space:
                tokenized_sentence += t.lemma_.lower() + " "  # Added a space after each token
        if tokenized_sentence!='':
            tokenized_sentences.append(tokenized_sentence.strip())  # Remove trailing space
    return tokenized_sentences


In [9]:
tokenized_texts=tokenize(sentence(extract_text_from_pdf(pdf_path)))

In [14]:
tokenized_texts

['natural consequence way living middle station life calculate kind virtue kind enjoyment peace plenty handmaid middle fortune temperance moderation quietness health society agreeable diversion desirable pleasure blessing attend middle station life way man go silently smoothly world comfortably embarrassed labor hand head sell life slavery daily bread harass perplexed circumstance rob soul peace body rest enrage passion envy secret burning lust ambition great thing easy circumstance slide gently world sensibly taste sweet live bitter feel happy learn day experience know sensibly',
 'press earnestly affectionate manner play young man precipitate misery nature station life bear provide necessity seek bread endeavor enter fairly station life recommend easy happy world mere fate fault hinder answer have discharge duty warn measure know hurt word kind thing stay settle home direct hand misfortune encouragement away',
 'close tell eld brother example earnest persuasion go low country war pre

In [15]:
import fitz  # PyMuPDF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(tokenized_texts)
def extract_text_and_font_sizes_from_pdf(pdf_path):
    document = fitz.open(pdf_path)
    text_font_size_data = []
    for page_num in range(len(document)):
        page = document.load_page(page_num)
        blocks = page.get_text("dict")["blocks"]
        for block in blocks:
            if block["type"] == 0:  # Text block
                for line in block["lines"]:
                    for span in line["spans"]:
                        text_font_size_data.append((span["text"], span["size"]))
    return text_font_size_data

def categorize_text_by_font_size(text_font_size_data):
    headings_threshold = 16
    subheadings_threshold = 12
    para_threshold = 8
    document_structure = {}
    current_heading = None
    current_subheading = None

    for text, font_size in text_font_size_data:
        if font_size > headings_threshold:
            if current_heading and not document_structure[current_heading]:
                document_structure[current_heading][current_heading] = []
            document_structure[text] = {}
            current_heading = text
            current_subheading = None
        elif font_size > subheadings_threshold:
            if current_heading:
                document_structure[current_heading][text] = []
                current_subheading = text
        elif font_size > para_threshold:
            if current_heading:
                if current_subheading:
                    document_structure[current_heading][current_subheading].append(text)
                else:
                    document_structure[current_heading][current_heading] = [text]
                    current_subheading = current_heading

    if current_heading and not document_structure[current_heading]:
        document_structure[current_heading][current_heading] = []

    return document_structure

def vectorize_text(document_structure):
    all_texts = []
    index_map = []

    for heading, subheadings in document_structure.items():
        all_texts.append(heading)
        index_map.append((heading, None, None))
        for subheading, texts in subheadings.items():
            all_texts.append(subheading)
            index_map.append((heading, subheading, None))
            for text in texts:
                all_texts.append(text)
                index_map.append((heading, subheading, text))
    return index_map


def find_best_match(tfidf_matrix, question_tfidf, index_map):
    similarities = cosine_similarity(question_tfidf, tfidf_matrix).flatten()
    best_index = np.argmax(similarities)
    best_similarity = similarities[best_index]
    best_match = index_map[best_index]
    
    return best_match, best_similarity, similarities

def get_best_paragraph(similarities, index_map, document_structure, threshold=0.2):
    best_heading, best_subheading, _ = index_map[np.argmax(similarities)]
    
    if best_subheading is None:
        all_text = ' '.join([text for subheadings in document_structure[best_heading].values() for text in subheadings])
    else:
        all_text = ' '.join(document_structure[best_heading][best_subheading])
    
    if np.max(similarities) < threshold:
        all_text = ' '.join([text for heading in document_structure for subheadings in document_structure[heading].values() for text in subheadings])
    
    return all_text

def filtered_context(pdf_path, question_tfidf, tfidf_matrix):
    text_font_size_data = extract_text_and_font_sizes_from_pdf(pdf_path)
    document_structure = categorize_text_by_font_size(text_font_size_data)
    
    '''print("Document structure:")
    for heading, subheadings in document_structure.items():
        print(f"Heading: {heading}")
        for subheading, texts in subheadings.items():
            print(f"  Subheading: {subheading}")
            for text in texts:
                print(f"    Text: {text}")'''
    
    index_map = vectorize_text(document_structure)
    best_match, best_similarity, similarities = find_best_match(tfidf_matrix, question_tfidf, index_map)
    
    print(f"Best match: {best_match}")
    print(f"Best similarity: {best_similarity}")
    
    best_paragraph = get_best_paragraph(similarities, index_map, document_structure)
    
    return best_paragraph
question='Where was the ship wrecked ?'
question_tokenized=tokenize([question])
question_tfidf=vectorizer.transform(question_tokenized)
answer = filtered_context(pdf_path, question_tfidf, tfidf_matrix)

print(f"Answer: {answer}")


Best match: ('3. Wrecked On A Desert Island ', '3. Wrecked On A Desert Island ', 'expedition, and bade Xury load both the others. As soon as he came fairly within my reach, I ')
Best similarity: 0.5214676256389748
Answer:   After this stop we made on to the southward continually for ten or twelve days, living very  sparing on our provisions, which began to abate very much, and going no oftener into the  shore than we were obliged to for fresh water. My design in this was to make the river  Gambia or Senegal — that is to say, anywhere about the Cape de Verde — where I was in  hopes to meet with some European ship; and if I did not, I knew not what course I had to  take, but to seek out for the lands, or perish there among the negroes. I knew that all the ships  from Europe, which sailed either to the coast of Guinea or to Brazil, or to the East Indies,  made this cape, or those islands; and in a word, I put the whole of my fortune upon this single  point, either that I must meet with so

In [11]:
def answer_question(contexts, question):
    qa_input = {
        'question': question,
        'context': contexts
    }
    result = question_answering_model(qa_input)
    
    return result

In [12]:
def top_answer(pdf_path, question, question_tfidf, tfidf_matrix):
    # Pair each element with its index
    contexts=filtered_context(pdf_path, question_tfidf, tfidf_matrix)
    print(question)
    return answer_question(contexts, question)


In [16]:
print(top_answer(pdf_path, question, question_tfidf, tfidf_matrix))

Best match: ('3. Wrecked On A Desert Island ', '3. Wrecked On A Desert Island ', 'expedition, and bade Xury load both the others. As soon as he came fairly within my reach, I ')
Best similarity: 0.5214676256389748
Where was the ship wrecked ?
{'score': 0.9783749580383301, 'start': 20817, 'end': 20830, 'answer': 'St.  Salvador'}
